## Project 3

This is a collaborative project conducted by the Fall 2017 students of DATA 620 at The City University of New York, in partial fulfillment of the requirements for the MS in Data Science degree.

### Problem Description

This is a Team Project! For this project, please work with the entire class as one collaborative group! Your project should be submitted (as an IPython Notebook via GitHub) by end of day on Monday, October 25th. The group should present their code and findings in our meet-up on Tuesday October 26th. The ability to be an effective member of a virtual team is highly valued in the data science job market.
Using any of the three classifiers described in chapter 6 of Natural Language Processing with Python, and any features you can think of, build the best name gender classifier you can. Begin by splitting the Names Corpus into three subsets: 500 words for the test set, 500 words for the dev-test set, and the remaining 6900 words for the training set. Then, starting with the example name gender classifier, make incremental improvements. Use the dev-test set to check your progress. Once you are satisfied with your classifier, check its final performance on the test set. How does the performance on the test set compare to the performance on the dev-test set? Is this what you'd expect?

Source: Natural Language Processing with Python, exercise 6.10.2.

### Contributors Include

* K. Joy Payton
* Keith Folsom
* Sonya Hong
* Shyam Balagurumurthy Viswanathan
* Derek Nokes
* Liam Byrne
* Latif Masud
* Valerie Briot


### First, Obtain the Corpus

Note: If not already executed, nltk.download() will allow you access to the names corpus

In [1]:
# Importing required libraries/packages
import nltk
from nltk.corpus import names
import random
import numpy as np
from nltk.metrics import *
import re
    

import string
from textstat.textstat import textstat
#nltk.download('names')

In [2]:
names = ([(name, 'male') for name in names.words('male.txt')] + \
         [(name, 'female') for name in names.words('female.txt')])

#names = random.shuffle(names)

In [3]:
random.shuffle(names)

# let's see what the randomly shuffles names look like
names[1:10]

[(u'Jeniffer', 'female'),
 (u'Laurie', 'male'),
 (u'Letta', 'female'),
 (u'Letty', 'female'),
 (u'Jasmina', 'female'),
 (u'Joya', 'female'),
 (u'Fonsie', 'male'),
 (u'Major', 'male'),
 (u'Joann', 'female')]

### Create three subsets for development and error analysis of the models.

##### Development set:
* 6900 names for the training set
* 500 names for the dev-test set  

##### Test set:
* 500 names for the testing set

We will first split the "names" data set into a Development set of 7,400 entries and a Test set of 500 entries.  
The Development set will be used to test each features as we build the module. This test will be split between a training set and a dev-test set.  

In [4]:
# Split the names set into a Development set and a Test set;
# Developmeent set will be used for training and testing each features as we build the model
#test_names, devtest_names, train_names = names[0:500], names[500:1000], names[1000:]
test_names, development_set_names = names[0:500], names[500:]

In [5]:
# Confirm the size of the three subsets
#print("Training Set = {}".format(len(train_names)))
#print("Dev-Test Set = {}".format(len(devtest_names)))
print("Development Set = {}".format(len(development_set_names)))
print("Test Set = {}".format(len(test_names)))

Development Set = 7444
Test Set = 500


### Feature Extractor Functions

This section below is to incrementallly improve the feature extraction functions which are subsequently applied to the development and test datasets.

We will make use of various documentation to identify various features; 

1. Last Letter
2. First Letter, most names begining with a vowel are associated with females  
3. Vowels count
4. Hard consonants using general rules of c and g
5. Soft consonants using general rules of c and g
6. Syllable Count of names via textstat 
7. Name length
8. Last two letters
9. Last 3 letters
10. Character count
11. Character present
12. Letter count??? --> need to check with Derek?
13. Letter pair present?? --> need to check with Derek?
14. First 2 letters
15. First letter, Last Letter, Last 2 Letters, last 3 Letters, 2-grams
16. ???

The features are coded below.  

We will use NaiveBayes and Decision tree classifier.


### Gender Identification Models building

To facilitate the analysis, a class for classifier was constructed. This class will have all the features and various methods; show error, display confusion matrix, evaluate fit of model.  

In [ ]:
class NLP_Classifier():
    def __init__(self,model):
        self.model = model
        #self.feat_num = feat_num
        
        #train_set = [(gender_features3(n), g)  for (n, g) in train_names]
        #devtest_set = [(gender_features3(n), g)  for (n, g) in devtest_names]
        #test_set = [(gender_features3(n), g)  for (n, g) in test_names]


    def get_features(self,name,feat_num):
        '''
        Parameters:
            name - string of name to extract feature
            feat_num - itterable collection of integers specifying features. *Defaults to 1:9 inclusive
                1: last letter
                2: first letter
                3: Vowel counts
                4: Hard consonant count
                5: Soft consonant count
                6: Syllable Count
                7: Name length
                8: Last two chars
                9: Last three chars
                10: char count --> feature for all alpha chars
                11: char present --> feature for all alpha chars (boolean)
        Returns:
            features: a dictionary of extracted features
        '''
        features = {}    
        
        # Converts feat_num to itterable if type is int
        if type(feat_num) is int:
            feat_num = (0, feat_num)        
       
        # Gender Feature 1: Last letter - book example
        if 1 in feat_num:
            features['last_letter'] = name[-1].lower()
            
        # Gender Feature 2: First letter - most names beginning with a vowel --> females
        if 2 in feat_num:
            features['first_letter'] = name[0].lower()
            
        # Gender Feature 3: Vowel Counts
        if 3 in feat_num:
            features['vowel_count'] = len(re.sub(r'[^aeiou]', '', name.lower()))
            
        # Gender Feature 4: Hard consonants using general rules of c and g
        if 4 in feat_num:
            features['hard_consts'] = len(re.findall(r'[cg][^eiy]', name.lower()))/2
            
        # Gender Feature 5: Soft consonants using general rules of c and g
        if 5 in feat_num:
            features['soft_consts'] = len(re.findall(r'[cg][eiy]', name.lower()))/2
            
        # Gender Feature 6: Syllable Count of names via textstat
        if 6 in feat_num:
            features['syllable_count'] = textstat.syllable_count(name.lower())
    
        # Gender Feature 7: Name length
        if 7 in feat_num:
            features["length"] = len(name)
        
        # Gender Feature 8: Last two chars
        if 8 in feat_num:
            features["last2letters"] = name[-2:].lower()
            
        # Gender Feature 9: Last three chars
        if 9 in feat_num:
            features["last3letters"] = name[-3:].lower()
    
        # Gender Feature 10: Char Counts (overfitts)
        if 10 in feat_num:
            for letter in string.ascii_lowercase:
                features["count_{0}".format(letter)] = name.lower().count(letter)
                
        # Gender Feature 11: Char Booleans (overfitts)
        if 11 in feat_num:
            for letter in string.ascii_lowercase:
                features["has_{0}".format(letter)] = letter in name.lower()
        
        
        if 12 in feat_num:
            features = {}
            letters=list(map(chr, range(ord('a'), ord('z') + 1)))
            for letter in letters:
                features["count(%s)" % letter] = name.lower().count(letter)


        if 13 in feat_num:
            features = {}
            letters=list(map(chr, range(ord('a'), ord('z') + 1)))
            for letter1 in letters:
                for letter2 in letters:
                    features["has("+letter1+letter2+")"] = (letter1+letter2 in name.lower())

        if 14 in feat_num:
            features["first2Letters"]=name[0:2].lower()

        if 15 in feat_num:
            features = {}
            features["firstletter"] = name[0].lower()
            features["lastletter"] = name[-1].lower()
            features["last2letter"] = name[-2:].lower()
            features["last3letter"] = name[-3:].lower()

            letters=list(map(chr, range(ord('a'), ord('z') + 1)))
            for letter1 in letters:
                features["count("+letter1+")"] = name.lower().count(letter1)
                features["has("+letter1+")"] = (letter1 in name.lower())
                # iterate over 2-grams
                for letter2 in letters:

                    features["has("+letter1+letter2+")"] = (letter1+letter2 in name.lower())


        if 16 in feat_num:
            # define features
            features = {}
            # has(fo) = True
            features["has(fo)"] = ('fo' in name.lower())
            # has(hu) = True
            features["has(hu)"] = ('hu' in name.lower())
            # has(rv) = True
            features["has(rv)"] = ('rv' in name.lower())    
            # has(rw) = True
            features["has(rw)"] = ('rw' in name.lower()) 
            # has(sp) = True
            features["has(sp)"] = ('sp' in name.lower())

            # lastletter = 'a'
            features["lastletter=a"] = ('a' in name[-1:].lower())
            # lastletter = 'f'
            features["lastletter=f"] = ('f' in name[-1:].lower())
            # lastletter = 'k'
            features["lastletter=k"] = ('k' in name[-1:].lower())

            # last2letter = 'ch'
            features["last2letter=ch"] = ('ch' in name[-2:].lower())
            # last2letter = 'do'
            features["last2letter=do"] = ('do' in name[-2:].lower())
            # last2letter = 'ia'
            features["last2letter=ia"] = ('ia' in name[-2:].lower())
            # last2letter = 'im'
            features["last2letter=im"] = ('im' in name[-2:].lower())
            # last2letter = 'io'
            features["last2letter=io"] = ('io' in name[-2:].lower())
            # last2letter = 'la'
            features["last2letter=la"] = ('la' in name[-2:].lower())
            # last2letter = 'ld'
            features["last2letter=ld"] = ('ld' in name[-2:].lower())
            # last2letter = 'na'
            features["last2letter=na"] = ('na' in name[-2:].lower())
            # last2letter = 'os'
            features["last2letter=os"] = ('os' in name[-2:].lower())
            # last2letter = 'ra'
            features["last2letter=ra"] = ('ra' in name[-2:].lower())
            # last2letter = 'rd'
            features["last2letter=rd"] = ('rd' in name[-2:].lower())
            # last2letter = 'rt'
            features["last2letter=rt"] = ('rt' in name[-2:].lower())
            # last2letter = 'sa'
            features["last2letter=sa"] = ('sa' in name[-2:].lower())
            # last2letter = 'ta'
            features["last2letter=ta"] = ('ta' in name[-2:].lower())
            # last2letter = 'us'
            features["last2letter=us"] = ('us' in name[-2:].lower())

            # last3letter = 'ana'
            features["last3letter=ana"] = ('ana' in name[-3:].lower())    
            # last3letter = u'ard'
            features["last3letter=ard"] = ('ard' in name[-3:].lower())        
            # last3letter = u'ita'
            features["last3letter=ita"] = ('ita' in name[-3:].lower())    
            # last3letter = u'nne'
            features["last3letter=nne"] = ('nne' in name[-3:].lower())    
            # last3letter = u'tta'
            features["last3letter=tta"] = ('tta' in name[-3:].lower())    
        
        return features

    
    def show_errors(self, errors, n=None):
        if n is not None: errors = errors[:n]  
        print("list of first %s errors :" %(n))
        for (tag, guess, name) in sorted(errors): 
            print('correct=%-8s guess=%-8s name=%-30s' %(tag, guess, name))
        print("")
        return None 
    
    
    def classifier_report(self,classifier,dataset,feat_num):
        feat_num = int(feat_num)
        dataset_predictions = [classifier.classify(self.get_features(n,feat_num))  for (n, g) in dataset]
        dataset_gold = [g  for (n, g) in dataset]
        cm=ConfusionMatrix(dataset_gold, dataset_predictions)
        print("Confusion Matrix: ")
        print(cm)
        print("")
    
    def fit_model(self,feat_num_start, feat_num):
       
        for i in np.arange(feat_num_start, feat_num+1):
            feat_num =int(i)
            errors = []
            
            # devtest-set and training set are constructed
            random.shuffle(development_set_names)
            devtest_names, train_names = development_set_names[0:500], development_set_names[500:]
            
            train_set = [(self.get_features(n,feat_num), g)  for (n, g) in train_names]
            devtest_set = [(self.get_features(n,feat_num), g)  for (n, g) in devtest_names]
            test_set = [(self.get_features(n,feat_num), g)  for (n, g) in test_names] 
            
            classifier = self.model.train(train_set) 
            
            # For errors list
            for (name, tag) in devtest_names:
                guess = classifier.classify(self.get_features(name,feat_num)) 
                if guess != tag: 
                    errors.append((tag, guess, name))    
                    
            #Print classifier report
            self.classifier_report(classifier,train_names,i)
            
            # Print errors        
            self.show_errors(errors, 5)
            
            # Print show_most_informative_features for NaiveBayes
            # Print pseudocode for DecisionTree
            if self.model ==nltk.NaiveBayesClassifier:
                classifier.show_most_informative_features(5)                
            elif self.model ==nltk.DecisionTreeClassifier:
                print(classifier.pseudocode(depth=5))
            
            print("-----------------------------------------")
            print("Accuracy of model {} using feature {} : {}".format(self.model,feat_num,nltk.classify.accuracy(classifier, devtest_set)))
            print("=========================================")
            print("")
        

###  NaiveBayes

The first classifier we selected is the Naive Bayes classifier.  

We will first use feature 1 - 14 (single features) and examine how the model perform for each. These results will provide us with the basis to derive more complexe features to refine model.  

In [7]:
clf = NLP_Classifier(nltk.NaiveBayesClassifier)
# specify initial and final feature
clf.fit_model(1, 14)

Confusion Matrix: 
       |    f      |
       |    e      |
       |    m    m |
       |    a    a |
       |    l    l |
       |    e    e |
-------+-----------+
female |<3579> 798 |
  male |  851<1716>|
-------+-----------+
(row = reference; col = test)


list of first 5 errors :
correct=female   guess=male     name=Gwendolen                     
correct=female   guess=male     name=Storm                         
correct=male     guess=female   name=Arne                          
correct=male     guess=female   name=Bruce                         
correct=male     guess=female   name=Randi                         

Most Informative Features
             last_letter = u'k'             male : female =     43.9 : 1.0
             last_letter = u'a'           female : male   =     37.1 : 1.0
             last_letter = u'f'             male : female =     13.3 : 1.0
             last_letter = u'p'             male : female =     12.6 : 1.0
             last_letter = u'm'             mal

Confusion Matrix: 
       |    f      |
       |    e      |
       |    m    m |
       |    a    a |
       |    l    l |
       |    e    e |
-------+-----------+
female |<3808> 589 |
  male | 1412<1135>|
-------+-----------+
(row = reference; col = test)


list of first 5 errors :
correct=male     guess=female   name=Ashley                        
correct=male     guess=female   name=Etienne                       
correct=male     guess=female   name=Lynn                          
correct=male     guess=female   name=Raj                           
correct=male     guess=female   name=Zelig                         

Most Informative Features
                 count_v = 2              female : male   =      9.5 : 1.0
                 count_w = 2                male : female =      5.2 : 1.0
                 count_a = 3              female : male   =      4.9 : 1.0
                 count_w = 1                male : female =      4.4 : 1.0
                 count_o = 2                mal

#### Evaluation of single features for Naive Baysien Classifier

Based on results for features 1-14, we can see that the best performing features are; 1, 8, 9, 10, 11, 13. Combining these features, we produced features 15 & 16.  

We will now evaluate classifier with these features.

In [8]:
clf.fit_model(15, 16)

Confusion Matrix: 
       |    f      |
       |    e      |
       |    m    m |
       |    a    a |
       |    l    l |
       |    e    e |
-------+-----------+
female |<3760> 631 |
  male |  500<2053>|
-------+-----------+
(row = reference; col = test)


list of first 5 errors :
correct=female   guess=male     name=Gill                          
correct=female   guess=male     name=Miriam                        
correct=male     guess=female   name=Julian                        
correct=male     guess=female   name=Noble                         
correct=male     guess=female   name=Rollin                        

Most Informative Features
             last2letter = u'na'          female : male   =     90.6 : 1.0
             last2letter = u'ra'          female : male   =     54.4 : 1.0
             last2letter = u'ia'          female : male   =     38.1 : 1.0
              lastletter = u'a'           female : male   =     38.0 : 1.0
                 has(fo) = True             mal

Based on these results, we would consider feature 15 to build the best model.

###  Decision Tree

The second classifier we selected is the Decision Tree classifier.  

Again, we will first use features 1 - 14 (single features) and examine how the model perform for each. These results will provide us with the basis to derive more complexe features to refine model.  

In [9]:
clf_dt = NLP_Classifier(nltk.DecisionTreeClassifier)
# specify initial and final feature
clf_dt.fit_model(1, 14)

Confusion Matrix: 
       |    f      |
       |    e      |
       |    m    m |
       |    a    a |
       |    l    l |
       |    e    e |
-------+-----------+
female |<3590> 802 |
  male |  840<1712>|
-------+-----------+
(row = reference; col = test)


list of first 5 errors :
correct=female   guess=male     name=Clo                           
correct=female   guess=male     name=Dell                          
correct=female   guess=male     name=Row                           
correct=male     guess=female   name=Connie                        
correct=male     guess=female   name=Godfree                       

-----------------------------------------
Accuracy of model <class 'nltk.classify.decisiontree.DecisionTreeClassifier'> using feature 1 : 0.748

Confusion Matrix: 
       |    f      |
       |    e      |
       |    m    m |
       |    a    a |
       |    l    l |
       |    e    e |
-------+-----------+
female |<4170> 206 |
  male | 2218 <350>|
-------+-----------+

Confusion Matrix: 
       |    f      |
       |    e      |
       |    m    m |
       |    a    a |
       |    l    l |
       |    e    e |
-------+-----------+
female |<4150> 219 |
  male | 1175<1400>|
-------+-----------+
(row = reference; col = test)


list of first 5 errors :
correct=female   guess=male     name=Agretha                       
correct=female   guess=male     name=Felipa                        
correct=male     guess=female   name=Barri                         
correct=male     guess=female   name=Lex                           
correct=male     guess=female   name=Reinhold                      

-----------------------------------------
Accuracy of model <class 'nltk.classify.decisiontree.DecisionTreeClassifier'> using feature 12 : 0.716



KeyboardInterrupt: 

Based on these results, it would seem that features; 1, 8, 9, 10, 12 lead to better outcomes.  We will now evaluate the evaluate the combination features.  

In [ ]:
clf_dt.fit_model(15, 16)

### Model Selection (Choose Best Candidate) 

Now that we have the best model build with both classifiers, we will test the model against the test data set and compare the results.  


#### Check the model's final performance on the test set. 

#### How does the performance on the test set compare to the performance on the dev-test set? 

#### Is this what you'd expect?